In [2]:
import numpy as np
import os
from collections import Counter
from ensemble import ensemble_defenses_util

/home/jsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [64]:
BS_data_FP="AE_mnist/legitimates/test_BS-mnist-clean.npy"
BS_label_FP="AE_mnist/legitimates/test_Label-mnist-clean.npy"
BS_pred_prob_FP="prediction_prob/pred_BS.npy"
BS_pred_logit_FP="prediction_logit/BS.npy"

BS = np.load(BS_data_FP)
labels = np.argmax(np.load(BS_label_FP), axis=1)
pred_prob = np.load(BS_pred_prob_FP)
pred_logit = np.load(BS_pred_logit_FP)

result_dir = "BB_data_preparation"
if not os.path.isdir(result_dir):
    os.mkdir(result_dir)
    
indices = {}
for i in range(10):
    ind_i= np.where(labels==i)[0]
    np.random.shuffle(ind_i)
    indices[str(i)] = ind_i
    
    
ind_sets = {}
query_budgets = [10, 50, 100, 500, 1000]
for budget in query_budgets:
    budget_per_class = budget//10
    ind_budget = indices["0"][:(budget-9*budget_per_class)]
    for label in range(1, 10):
        ind_budget = np.hstack((ind_budget, indices[str(label)][:budget_per_class]))
    np.random.shuffle(ind_budget)
    ind_sets[str(budget)] = ind_budget
    print("Budget {}: {}".format(budget, Counter(labels[ind_budget]).values()))
    np.save(
        os.path.join(result_dir, "budget{}_train_set.npy".format(budget)),
        BS[ind_budget])
np.save(os.path.join(result_dir, "train_set_ori_indices.npy"), ind_sets['1000'])
val_ind_sets = np.array([ind for ind in range(10000) if ind not in ind_sets[str(1000)]])
np.save(os.path.join(result_dir, "validation_set_9k_data.npy"), BS[val_ind_sets])
np.save(os.path.join(result_dir, "validation_set_9k_label.npy"), labels[val_ind_sets])

Budget 10: dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Budget 50: dict_values([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
Budget 100: dict_values([10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
Budget 500: dict_values([50, 50, 50, 50, 50, 50, 50, 50, 50, 50])
Budget 1000: dict_values([100, 100, 100, 100, 100, 100, 100, 100, 100, 100])


In [65]:
BB_labels_dir=os.path.join(result_dir, "BB_labels")
os.mkdir(BB_labels_dir)
ensembles = ["RD", "MV", "AVEP", "T2MV", "AVEL"]
for ensembleID in range(4):
    cur_labels_dir = os.path.join(BB_labels_dir, ensembles[ensembleID])
    os.mkdir(cur_labels_dir)
    for budget in query_budgets:
        cur_labels = ensemble_defenses_util(pred_prob[:, ind_sets[str(budget)], :], ensembleID)
        np.save(os.path.join(cur_labels_dir, "{}.npy".format(budget)), cur_labels)
    
cur_labels_dir = os.path.join(BB_labels_dir, "AVEL")
os.mkdir(cur_labels_dir)
ensembleID = 2 # averageing approach
for budget in query_budgets:
    cur_labels = ensemble_defenses_util(pred_logit[:, ind_sets[str(budget)], :], ensembleID)
    np.save(os.path.join(cur_labels_dir, "{}.npy".format(budget)), cur_labels)

In [66]:
ind_budget = indices["0"][100:200]
for label in range(1, 10):
    ind_budget = np.hstack((ind_budget, indices[str(label)][100:200]))
    np.random.shuffle(ind_budget)
    
print("Budget {}: {}".format("second 1000", Counter(labels[ind_budget]).values()))
np.save(os.path.join(result_dir, "BS_1k_data_For_AE.npy"), BS[ind_budget])
np.save(os.path.join(result_dir, "BS_1k_label_For_AE.npy"), labels[ind_budget])

Budget second 1000: dict_values([100, 100, 100, 100, 100, 100, 100, 100, 100, 100])


In [3]:
# Collect labels when targeting the undefended model
result_dir = "BB_data_preparation"
BB_labels_dir = os.path.join(result_dir, "BB_labels")
cur_labels_dir = os.path.join(BB_labels_dir, "UM")

from tensorflow.keras.models import load_model
ori_model = load_model("./models_mnist/model-mnist-cnn-clean.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
query_budgets = [10, 50, 100, 500, 1000]
for budget in query_budgets:
    X = np.load(os.path.join(result_dir, "budget{}_train_set.npy".format(budget)))
    Y = ori_model.predict(X)
    Y = np.argmax(Y, axis=1)
    np.save(os.path.join(cur_labels_dir, "{}.npy".format(budget)), Y)